In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext("local", "Customer Spendings App")
spark = SparkSession.builder.appName("Customer Spendings App").getOrCreate()

In [2]:
foods_rdd = sc.textFile("Foods.csv").map(lambda x: x.split(","))

In [6]:
foods_rdd=sc.parallelize(foods_rdd.collect()[1:])

In [9]:
sales_rdd = sc.textFile("Sales.csv").map(lambda x: x.split(","))

In [10]:
sales_rdd=sc.parallelize(sales_rdd.collect()[1:])

1)How many of the customers have spent more than 10$ in the restaurant

In [13]:
foods_dict_rdd = foods_rdd.map(lambda x: (x[0], float(x[2]))).collectAsMap()

In [15]:
customer_spendings_rdd = sales_rdd.map(lambda x: (x[0], foods_dict_rdd[x[1]])).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 10)

In [17]:
num_customers = customer_spendings_rdd.count()

In [18]:
print("Number of customers who have spent more than $10:", num_customers)

Number of customers who have spent more than $10: 84


2)How many customers had more than 3 transactions?

In [19]:
customer_transactions_rdd = sales_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 3)

In [20]:
customer_transactions_rdd.collect()

[('77', 4)]

In [21]:
num_customers = customer_transactions_rdd.count()

In [22]:
print("Number of customers who have made more than 3 transactions:", num_customers)

Number of customers who have made more than 3 transactions: 1


3.Which food item has been consumed the most?

In [23]:
food_count_rdd = sales_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)

In [25]:
food_name_count_rdd = foods_rdd.map(lambda x: (x[0], x[1])).join(food_count_rdd)

In [26]:
sorted_food_rdd = food_name_count_rdd.map(lambda x: (x[1][1], x[1][0])).sortByKey(False)

In [27]:
most_consumed_food = sorted_food_rdd.first()

In [28]:
print("Most consumed food item: ", most_consumed_food[1], " (", most_consumed_food[0], " times)")

Most consumed food item:  Donut  ( 35  times)


4. Which food item is preferred by male and by female?

In [29]:
customers_rdd = sc.textFile("customers.csv").map(lambda line: line.split(","))

In [30]:
customers_rdd=sc.parallelize(customers_rdd.collect()[1:])

In [33]:
foods_dict_rdd = foods_rdd.map(lambda x: (x[0], x[1])).collectAsMap()

In [45]:
sales_customer_rdd = sales_rdd.map(lambda x: (x[0], (x[1]))).join(customers_rdd.map(lambda x: (x[0], (x[3]))))

In [47]:
male_sales_rdd = sales_customer_rdd.filter(lambda x: x[1][1] == 'Male').map(lambda x: (x[1][0], 1)).reduceByKey(lambda x, y: x + y)


In [49]:
female_sales_rdd = sales_customer_rdd.filter(lambda x: x[1][1] == 'Female').map(lambda x: (x[1][0], 1)).reduceByKey(lambda x, y: x + y)

In [58]:
male_sales_rdd.values().max()

15

In [55]:
item_male=male_sales_rdd.filter(lambda x:x[1]==15)

In [56]:
#BURITO IS CONSUMED MOSTYLY BY MALE
item_male.collect()

[('2', 15)]

In [59]:
female_sales_rdd.values().max()

27

In [60]:
item_female=female_sales_rdd.filter(lambda x:x[1]==27)

In [62]:
#DONE IS CONSUMED MOSTYLY BY MALE
item_female.collect()

[('9', 27)]

 5. How many records are available in each of the file?

In [65]:
print("Foods count",foods_rdd.count())
print("Customer count",customers_rdd.count())
print("Sales count",sales_rdd.count())

Foods count 10
Customer count 1000
Sales count 250


6. Which profession of the people most likely visit the restaurant the most

In [88]:
joined_rdd =customers_rdd.map(lambda x: (x[0], (x[4], x[5]))).join(sales_rdd.map(lambda x: (x[0], x[1])))

In [89]:
occupation_count_rdd = joined_rdd.map(lambda x: (x[1][0][1], 1))

In [91]:
occupation_count_rdd = occupation_count_rdd.reduceByKey(lambda x, y: x + y)

In [92]:
most_frequent_occupation = occupation_count_rdd.max(key=lambda x: x[1])[0]

In [93]:
print("The profession of people most likely to visit the restaurant the most is:", most_frequent_occupation)

The profession of people most likely to visit the restaurant the most is: Marketing Manager


 7. Which company visits restaurant the most ?


In [102]:
company_count_rdd = joined_rdd.map(lambda x: (x[1][0], 1)).reduceByKey(lambda x, y: x + y)

In [105]:
most_frequent_company = company_count_rdd.max(key=lambda x: x[1])[0][0]

In [106]:
print("The company that visits the restaurant the most is:", most_frequent_company)

The company that visits the restaurant the most is: Dabvine


8 .Which customer spends the most in the restaurant

In [110]:
customer_spending_rdd = sales_rdd.reduceByKey(lambda x, y: x + y)

In [112]:
top_spender = customer_spending_rdd.max(key=lambda x: x[1])[0]

In [115]:
top_spender

'147'

In [119]:
cust_name=customers_rdd.filter(lambda x:x[0]==top_spender).collect()

In [120]:
cust_name

[['147', 'Theresa', 'Morales', 'Female', 'Abata', 'Occupational Therapist']]

In [122]:
print("The customer who has spent the most is:", cust_name[0][1] + " " + cust_name[0][2])

The customer who has spent the most is: Theresa Morales
